In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

print(f'Tensorflow version: {tf.__version__}')

In [ ]:
happines_data_2015 = pd.read_csv('../input/world-happiness/2015.csv', parse_dates=True, encoding = "cp1252")
happines_data_2016 = pd.read_csv('../input/world-happiness/2016.csv', parse_dates=True, encoding = "cp1252")
happines_data_2017 = pd.read_csv('../input/world-happiness/2017.csv', parse_dates=True, encoding = "cp1252")
happines_data_2018 = pd.read_csv('../input/world-happiness/2018.csv', parse_dates=True, encoding = "cp1252")
happines_data_2019 = pd.read_csv('../input/world-happiness/2019.csv', parse_dates=True, encoding = "cp1252")

In [ ]:
happines_data_2015.drop(['Region', 'Standard Error', 'Dystopia Residual'], axis=1, inplace=True)
happines_data_2015.rename({'Happiness Rank':'Overall rank', 
                           'Country':'Country or region', 
                           'Happiness Score': 'Score', 
                           'Economy (GDP per Capita)': 'GDP per capita', 
                           'Family':'Social support', 
                           'Health (Life Expectancy)':'Healthy life expectancy', 
                           'Freedom':'Freedom to make life choices', 
                           'Trust (Government Corruption)':'Perceptions of corruption'}, axis=1, inplace=True)
happines_data_2015.head()

In [ ]:
happines_data_2016.drop(['Region', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Dystopia Residual'], axis=1, inplace=True)
happines_data_2016.rename({'Happiness Rank':'Overall rank', 
                           'Country':'Country or region', 
                           'Happiness Score': 'Score', 
                           'Economy (GDP per Capita)': 'GDP per capita', 
                           'Family':'Social support', 
                           'Health (Life Expectancy)':'Healthy life expectancy', 
                           'Freedom':'Freedom to make life choices', 
                           'Trust (Government Corruption)':'Perceptions of corruption'}, axis=1, inplace=True)
happines_data_2016.head()

In [ ]:
happines_data_2017.drop(['Whisker.high', 'Whisker.low', 'Dystopia.Residual'], axis=1, inplace=True)
happines_data_2017.rename({'Happiness.Rank':'Overall rank', 
                           'Country':'Country or region', 
                           'Happiness.Score': 'Score', 
                           'Economy..GDP.per.Capita.': 'GDP per capita', 
                           'Family':'Social support', 
                           'Health..Life.Expectancy.':'Healthy life expectancy', 
                           'Freedom':'Freedom to make life choices', 
                           'Trust..Government.Corruption.':'Perceptions of corruption'}, axis=1, inplace=True)
happines_data_2017.head()

In [ ]:
happines_data = pd.concat([happines_data_2019, happines_data_2018, happines_data_2017, happines_data_2016, happines_data_2015])
happines_data.drop(['Overall rank', 'Country or region'], axis=1, inplace=True)
happines_data.head(800)

In [ ]:
happines_data.describe()

In [ ]:
happines_data['Perceptions of corruption'].isnull().values.any()

In [ ]:
happines_data['Perceptions of corruption'].isnull().sum()

In [ ]:
happines_data['Perceptions of corruption'] = happines_data['Perceptions of corruption'].fillna(0)

In [ ]:
X = happines_data[['GDP per capita',
                   'Social support', 
                   'Healthy life expectancy',
                   'Freedom to make life choices',
                   'Generosity',
                   'Perceptions of corruption']]

y = happines_data['Score']

In [ ]:
# corr = happines_data.corr(method = "pearson")
corr = happines_data.corr(method = "spearman")
# corr = happines_data.corr(method = "kendall")

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True, ax=ax, annot=True,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=12, input_shape=(X_train.shape[1],), activation='relu'),
  tf.keras.layers.Dense(units=52, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(units=120, activation='exponential'),
  tf.keras.layers.Dense(units=34, activation='relu'),
  tf.keras.layers.Dense(units=14, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

reg = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=15, callbacks=[callback])

In [ ]:
fig, ax = plt.subplots(figsize=(25,8)) 

plt.plot(reg.history['mse'], label='mean_squared_error')
plt.plot(reg.history['val_mse'], label='val_mean_squared_error', linestyle='--')
plt.plot(reg.history['mae'], label='mean_absolute_error')
plt.plot(reg.history['val_mae'], label='val_mean_absolute_error', linestyle='--')
plt.legend()

In [ ]:
correct=[]
X_list=X_test
y_list=list(y_test)
for i in range(len(X_list)):
    a=y_list[i]
    b=model.predict([[X_list[i][0],X_list[i][1],X_list[i][2],X_list[i][3],X_list[i][4],X_list[i][5]]])[0][0]

    correct.append((abs(a-b)/a))
#     print(f'{i} => {a:.4f} - {b:.4f} - {(abs(a-b)/a):.4f}')

print(1-sum(correct) / len(correct))

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout

style = {'description_width': '170px'}
layout = Layout(width='600px')

gdp = widgets.FloatSlider(min=0, max=2.1, step=0.01, value=1, description='GDP per capita', style=style, layout=layout)
social = widgets.FloatSlider(min=0, max=1.7, step=0.01, value=0.5, description='Social support', style=style, layout=layout)
health = widgets.FloatSlider(min=0, max=1.2, step=0.01, value=0.5, description='Healthy life expectancy', style=style, layout=layout)
freedom = widgets.FloatSlider(min=0, max=0.7, step=0.01, value=0.5, description='Freedom to make life choices', style=style, layout=layout)
gen = widgets.FloatSlider(min=0, max=0.6, step=0.01, value=0.3, description='Generosity', style=style, layout=layout)
cor = widgets.FloatSlider(min=0, max=0.5, step=0.01, value=0.2, description='Perceptions of corruption', style=style, layout=layout)

def f(gdp, social, health, freedom, gen, cor):
    print(f'Predicted value: {model.predict([[gdp, social, health, freedom, gen, cor]])[0][0]:.5f}')

    
out = widgets.interactive_output(f, {'gdp': gdp, 'social': social, 'health': health, 'freedom': freedom, 'gen': gen, 'cor': cor,})

widgets.HBox([widgets.VBox([gdp, social, health, freedom, gen, cor]), out])